In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import sklearn
###

1) 데이터 로드 및 분할하기

In [7]:
df = pd.read_csv("final_data.csv", index_col=0)
df.head()

,Sentence,Change
0,"주가 사흘째 상승세\n아남텔레콤, PDA 데이터 전송서비스 개시\n<회전목마> 삼성...",0
1,주가 3백70선 회복\n`반도체업체는 여전히 안정된 직장'\n휴대형 입체영상 모니터...,0
2,"삼성디자인연구원, 전통유산 디자인 CD-롬 제작\n주가 3백50대로 밀려\n<확대경...",1
3,"한도폐지 첫날 주가 하락세\n주가, 11년만에 최저치로 폭락\n",0
4,"주가 급락세 지속, 3백10선도 위협\n삼성전자, 매출 급신장\n엘니뇨 덕, 에어컨...",1


In [8]:
df_shuffled=sklearn.utils.shuffle(df)
df_shuffled.head()

,Sentence,Change
1572,[유럽DR]삼성전자 홀로 상승\n[유럽DR-마감] 뉴욕증시 약세 불구 대부분 상승\...,1
5291,"[마감시황]코스피 오늘도 상승…2160선 되찾아\n갤럭시폴드, 40만원 내려서 판다...",1
4398,100대 기업 등기임원 58년생 가장 많아\n푸른 창공! 무인 비행 드론페스티벌\n...,1
4706,"코스피 美증시 약세에 하락…외인 '팔자'\n중국, 반도체 사업 정부 투자로 '자신만...",1
1923,"FT 세계 500대 기업, 삼성전자 35위에 올라\n이현봉 삼성 가전사장 ""가을에 ...",0


In [10]:
train_ratio = 0.8
train_range = int(len(df_shuffled) * train_ratio)
train_range

4721

In [11]:
train_data = df_shuffled[:train_range]
test_data = df_shuffled[train_range:]

print(len(train_data), len(test_data))

4721 1181


2) 데이터 정제하기

In [ ]:
'''
document 열의 중복 제거 (document와 label로만 구성)
'''
train_data.drop_duplicates(subset=['document'], inplace=True)

In [ ]:
'''
label값 분포 확인
'''
train_data['label'].value_counts().plot(kind = 'bar')

In [ ]:
'''
Null이 있나 확인 후 있으면 제거
'''
print(train_data.isnull().values.any())
print(train_data.isnull().sum())
train_data.loc[train_data.document.isnull()]
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
'''
한글과 공백을 제외하고 모두 제거
'''
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data[:5]

In [ ]:
'''
한글이 아닌 리뷰는 empty가 됐을 것으므로, 이를 Null로 변경
'''
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

In [ ]:
train_data.loc[train_data.document.isnull()] # Null 있는 행 출력 

In [ ]:
train_data = train_data.dropna(how = 'any')
print(len(train_data))

In [ ]:
'''
테스트 데이터도 동일하게 전처리 진행
'''

test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

3) 토큰화

In [ ]:
'''
Okt는 위와 같이 KoNLPy에서 제공하는 형태소 분석기임
한국어을 토큰화할 때는 영어처럼 띄어쓰기 기준으로 토큰화를 하는 것이 아니라, 주로 형태소 분석기를 사용함
stem = True를 사용하면 일정 수준의 정규화를 수행해주는데, 예를 들어 '이런'이 '이렇다'로 변환되고, '만드는'이 '만들다'로 변환됨
'''
okt = Okt()

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

In [ ]:
print(X_train[:3])

In [ ]:
'''
테스트 데이터도 동일하게 토큰화 진행
'''
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

4) 정수 인코딩

In [ ]:
'''
훈련데이터에 대해서 단어집합(vocaburary) 만들기
'''
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
print(tokenizer.word_index) # 각 단어에 고유한 정수 부여

In [ ]:
'''
등장 빈도수가 3회 미만인 단어 비중 확인
'''
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
'''
전체 단어 개수 중 빈도수 2이하인 단어는 제거
'''
vocab_size = total_cnt - rare_cnt + 1 # 0번 패딩 토큰을 고려하여 + 1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
'''
단어 집합 크기를 반영해서 다시 정수 배정
'''
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

5) 빈 샘플 제거

In [ ]:
'''
전체 데이터에서 빈도수가 낮은 단어가 삭제되어, 빈도수가 낮은 단어만으로 구성된 샘플이 있으면
empty sample이 되므로 이를 제거해 주는 과정
'''

drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

6) 패딩

In [ ]:
'''
서로 다른 샘플들의 길이를 동일하게 맞춰주는 작업
'''
print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
'''
전체 샘플 중 길이가 max_len 이하인 샘플의 비율을 확인
'''
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
MAX_LEN = 30
below_threshold_len(MAX_LEN, X_train)

In [ ]:
'''
샘플의 길이를 MAX_LEN에 맞춤
'''
X_train = pad_sequences(X_train, maxlen=MAX_LEN)
X_test = pad_sequences(X_test, maxlen=MAX_LEN)